# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [49]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [50]:
# checking the current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [51]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

### Let's check the number of rows in the csv file

In [52]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [53]:
full_data_rows_list[0]

['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner',
 'Logged In',
 'Mohammad',
 'M',
 '0',
 'Rodriguez',
 '277.15873',
 'paid',
 'Sacramento--Roseville--Arden-Arcade, CA',
 'PUT',
 'NextSong',
 '1.54051E+12',
 '961',
 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)',
 '200',
 '1.54328E+12',
 '88']

In [54]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [55]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
                        WITH REPLICATION =
                        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                   )

except Exception as e:
    print(e)

#### Set Keyspace

In [56]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

## Query 1
### with Apache Cassandra we model the database tables on the queries you want to run so, let's build our table that have all columns needed to bulid the query
### It's always better to state the query in a raw cell before starting to have a clear look and take the right steps

#### Now, let's create the table with all the attributes we will be needing to run the requested query as follow. Columns as following:
    session_id
    item_in_session
    artist_name,
    song_title,
    song_length

In [57]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_item_session"
query = query + "(session_id int, item_in_session int, artist_name text, song_length float, song_title text, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [58]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_playlist_item_session(session_id, item_in_session, artist_name, song_length, song_title)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), float(line[5]), str(line[9])))

#### let's verify that the data was entered into the table by testing the SELECT statement

In [59]:
query= """
SELECT
    artist_name,
    song_title,
    song_length
    
FROM
    song_playlist_item_session
WHERE
    session_id = 338
AND
    item_in_session = 4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

## Query 2
### I will be following the same steps I've done in the first query, writing the query in a raw cell, creating the table, inserting the data into the table, and finally testing my work.

In [60]:
query = "CREATE TABLE IF NOT EXISTS user_activity_item_session"
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [61]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_activity_item_session(user_id, session_id, item_in_Session, artist_name, song_title, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))

In [62]:
query = """
SELECT
    artist_name,
    song_title,
    user_first_name,
    user_last_name
    
FROM
    user_activity_item_session
WHERE
    user_id = 10
AND
    session_id = 182
    
"""
    
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 3
#### Again, I will be following the same steps as in the previous questions..

In [68]:
query = "CREATE TABLE IF NOT EXISTS user_activity_song"
query = query + "(song_title text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY ((song_title), user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [69]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_activity_song(song_title, user_id, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

In [74]:
query = """
SELECT
    user_first_name,
    user_last_name
FROM
    user_activity_song
WHERE
    song_title ='All Hands Against His Own'
"""
    
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [76]:
try:    
    session.execute("DROP TABLE song_playlist_item_session")
    session.execute("DROP TABLE user_activity_item_session")
    session.execute("DROP TABLE user_activity_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [77]:
session.shutdown()
cluster.shutdown()